In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")


In [2]:
df = pd.read_csv("Car_Sales_Cleaned_Dataset.csv")
df

,Year_of_Purchase,Insurance_year,Insurance_month,Insurance_day,Fuel_type_le,Transmission_le,RTO_le,Insurance_Type_le,Price_N_F,Rating_N_b_F,Kilometers_N_F,Owner_oe_N_F
0,2010,2021,7,16,3,1,10,6,20.444317,23.437822,186.692260,0.000000
1,2013,2021,8,6,3,1,26,0,24.040101,27.300178,198.849189,1.000000
2,2014,2021,4,28,3,1,26,0,24.378381,29.381573,152.423751,1.000000
3,2013,2022,1,1,3,1,14,0,22.085038,31.566616,166.577309,0.000000
4,2017,2021,7,16,3,1,642,1,23.417094,31.566616,110.625494,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
32153,2008,2021,7,16,3,1,72,6,19.679897,14.066117,309.502827,1.414214
32154,2015,2021,7,16,0,1,467,6,22.360680,15.412467,233.463059,1.000000
32155,2008,2021,7,16,0,1,191,6,26.591479,16.841604,401.649101,1.000000
32156,2016,2019,10,27,0,1,494,5,27.831577,38.770013,228.048241,0.000000


In [3]:
X = df.drop(columns = ["Price_N_F"])
y=df["Price_N_F"]

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(X,y,train_size=0.8,random_state=42)

In [5]:
from sklearn.tree import DecisionTreeRegressor,plot_tree
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [6]:
model  = DecisionTreeRegressor(random_state = 42)
model.fit(x_train,y_train)

DecisionTreeRegressor(random_state=42)

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


In [8]:
from sklearn.tree import plot_tree
#plt.figure(figsize =(20,20),dpi=300)
#plot_tree(model,filled=True,feature_names=x_train.columns, rounded=True)
plt.show()


In [9]:
#hyper parameter tuning
estimator =DecisionTreeRegressor(random_state=42)
param_grid = {"criterion":["gini","squared_error"],
              "max_depth":list(range(1,16))}
dt_grid = GridSearchCV(estimator,param_grid,scoring="r2",cv=5)
dt_grid.fit(x_train,y_train)
dt = dt_grid.best_estimator_
dt

DecisionTreeRegressor(max_depth=7, random_state=42)

In [10]:
# identifing the import features 
feats_ab = pd.DataFrame(data=dt.feature_importances_,
                       index=x_train.columns,
                   columns =["importances"] )
important_feature_dt = feats_ab[feats_ab["importances"]>0].index.tolist()
important_feature_dt

['Year_of_Purchase',
 'Insurance_year',
 'Insurance_month',
 'Insurance_day',
 'Fuel_type_le',
 'Transmission_le',
 'RTO_le',
 'Insurance_Type_le',
 'Rating_N_b_F',
 'Kilometers_N_F',
 'Owner_oe_N_F']

In [11]:
X_train_dt = x_train[important_feature_dt]
X_test_dt = x_test[important_feature_dt]
# modelling
dt =dt_grid.best_estimator_
dt.fit(X_train_dt,y_train)
# evaluation
ypred_train = dt.predict(X_train_dt)
ypred_test = dt.predict(X_test_dt)
print("train accuracy:",r2_score(y_train,ypred_train))
print("CVscore:",cross_val_score(dt,X_train_dt,y_train,cv=5,scoring = "r2").mean())
print("testaccuracy:",r2_score(y_test,ypred_test))

train accuracy: 0.6951812595428931
CVscore: 0.6716699144065045
testaccuracy: 0.6682525013305232


In [12]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [34]:
# hyper parameter tuning 
estimator = RandomForestRegressor(random_state=42)
param_grid = {"n_estimators":list(range(50,60))}
rf_grid  =GridSearchCV(estimator,param_grid,scoring ="r2",cv=5)
rf_grid.fit(x_train,y_train)
rf=rf_grid.best_estimator_
rf

RandomForestRegressor(n_estimators=59, random_state=42)

In [35]:
# import features selection in the data set
feats_ab = pd.DataFrame(data=rf.feature_importances_,
                       index=x_train.columns,
                       columns= ["importance"])
important_features_rf = feats_ab[feats_ab["importance"]>0].index.tolist()
important_features_rf

['Year_of_Purchase',
 'Insurance_year',
 'Insurance_month',
 'Insurance_day',
 'Fuel_type_le',
 'Transmission_le',
 'RTO_le',
 'Insurance_Type_le',
 'Rating_N_b_F',
 'Kilometers_N_F',
 'Owner_oe_N_F']

In [36]:
X_train_rf = x_train[important_features_rf]
X_test_rf = x_test[important_features_rf]
 # modelling
rf = rf_grid.best_estimator_
rf.fit(X_train_rf,y_train)

# evaluation
ypred_train =rf.predict(X_train_rf) 
ypred_test =rf.predict(X_test_rf)


print("train_accuracy:",r2_score(y_train,ypred_train))
print("CVscore:",cross_val_score(rf,X_train_rf,y_train,cv=5,scoring="r2").mean())
print("test_accuracy:",r2_score(y_test,ypred_test))

train_accuracy: 0.9558901840477028
CVscore: 0.6882518423416923
test_accuracy: 0.6873049746341588


In [9]:
from xgboost import XGBRFRegressor
from sklearn.model_selection import GridSearchCV

In [20]:
# hyper parameter tuning
estimator = XGBRFRegressor()
param_grid = {"n_estimators":[60,55],
              "max_depth":[3,6,5],
              "gamma":[0,0.5,0.1]}
xgb_grid = GridSearchCV(estimator,param_grid,scoring="r2",cv=5)
xgb_grid.fit(x_train,y_train)

xgb = xgb_grid.best_estimator_
xgb

XGBRFRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bytree=None, device=None,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, feature_types=None, gamma=0, grow_policy=None,
               importance_type=None, interaction_constraints=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=6, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=55, n_jobs=None,
               num_parallel_tree=None, objective='reg:squarederror',
               random_state=None, reg_alpha=None, ...)

In [21]:
# important features selection
feats_xgb  = pd.DataFrame(data=xgb.feature_importances_,
                         index=x_train.columns,
                         columns=["importance"])
importance_features_xgb = feats_xgb[feats_xgb["importance"]>0].index.tolist()
importance_features_xgb

['Year_of_Purchase',
 'Insurance_year',
 'Insurance_month',
 'Insurance_day',
 'Fuel_type_le',
 'Transmission_le',
 'RTO_le',
 'Insurance_Type_le',
 'Rating_N_b_F',
 'Kilometers_N_F',
 'Owner_oe_N_F']

In [24]:
X_train_xgb = x_train[importance_features_xgb]
X_test_xgb = x_test[importance_features_xgb]

# modellling 
xbg = xgb_grid.best_estimator_
xgb.fit(X_train_xgb,y_train)

# evaluation
ypred_train = xgb.predict(X_train_xgb)
ypred_test  = xgb.predict(X_test_xgb)

print("trainaccuracy:",r2_score(y_train,ypred_train))
print("cv score:",cross_val_score(xgb,X_train_xgb,y_train,cv = 5,scoring="r2").mean())
print("testaccuracy:",r2_score(y_test,ypred_test))

trainaccuracy: 0.6957969560707883
cv score: 0.6861243323753368
testaccuracy: 0.6823775419168085


# adaboosting

In [20]:
from sklearn.ensemble import AdaBoostRegressor
estimator = AdaBoostRegressor(random_state=42)
param_grid = {"n_estimators":list(range(1,20))}

ab_grid = GridSearchCV(estimator,param_grid,scoring="r2",cv=5)
ab_grid.fit(x_train,y_train)
ab = ab_grid.best_estimator_
ab

AdaBoostRegressor(n_estimators=3, random_state=42)

In [21]:
# import feartures selection in the data set
feats_ab = pd.DataFrame(data = ab.feature_importances_,
                       index = x_train.columns,
                       columns = ["importance"])
important_features_ab = feats_ab[feats_ab["importance"]>0].index.tolist()
important_features_ab

['Year_of_Purchase', 'Fuel_type_le', 'Transmission_le', 'Rating_N_b_F']

In [22]:
X_train_ab = x_train[important_features_ab]
X_test_ab = x_test[important_features_ab]

# modelling
ab = ab_grid.best_estimator_
ab.fit(X_train_ab,y_train)
# evaluation
ypred_train = ab.predict(X_train_ab)
ypred_test = ab.predict(X_test_ab)

print("train_accuracy:",r2_score(y_train,ypred_train))
print("cvscore:",cross_val_score(ab,X_train_ab,y_train,cv=5,scoring="r2").mean())
print("testaccuracy:",r2_score(y_test,ypred_test))

train_accuracy: 0.5917725678971588
cvscore: 0.6073409752286161
testaccuracy: 0.5759020226235694


In [31]:
from sklearn.ensemble import GradientBoostingRegressor
estimator = GradientBoostingRegressor()
param_grid = {"n_estimators":list(range(1,10)),
               "learning_rate":[0.1,0.2,0.3,0.4,0.5]}
gb_grid = GridSearchCV(estimator,param_grid,scoring = "r2",cv= 5)
gb_grid.fit(x_train,y_train)
gb = gb_grid.best_estimator_
gb

GradientBoostingRegressor(learning_rate=0.5, n_estimators=9)

In [32]:
# import feature selection
feats_gb = pd.DataFrame(data=gb.feature_importances_,
                       index =x_train.columns,
                       columns=["importance"])
import_features_gb = feats_gb[feats_gb["importance"]>0].index.tolist()
import_features_gb

['Year_of_Purchase',
 'Fuel_type_le',
 'Transmission_le',
 'RTO_le',
 'Insurance_Type_le',
 'Rating_N_b_F',
 'Kilometers_N_F']

In [33]:
X_train_gb = x_train[import_features_gb]
X_test_gb = x_test[import_features_gb]


gb = gb_grid.best_estimator_
gb.fit(X_train_gb,y_train)

ypred_train = gb.predict(X_train_gb)
ypred_test = gb.predict(X_test_gb)

print("trainaccuracy:",r2_score(y_train,ypred_train))
print("cv score:",cross_val_score(gb,X_train_gb,y_train,cv =5,scoring ="r2").mean())
print("testaccuracy:",r2_score(y_test,ypred_test))

trainaccuracy: 0.6981546432910983
cv score: 0.6920239138065183
testaccuracy: 0.6889989785525338
